# FIFA + RS

In [ ]:
import random
import math
import copy
import numpy as np
from time import time


def read_bpp_file(file_path):
    with open(file_path, 'r') as f:
        n = int(f.readline().strip())
        capacity = int(f.readline().strip())
        objects = [int(line.strip()) for line in f]
    return (n, capacity, objects)


class Item:
    def __init__(self, label: int = None, size: int = None):
        self._label = label  # index
        self._size = size  # weight


class Bin:
    def __init__(self, items: list = None, capacity: int = 0):
        if items is None:
            items = []
        self._items = items
        self._capacity = capacity

    def add_item(self, item: Item):
        self._items.append(item)
        self._capacity += item._size

    def get_labels(self) -> list[int]:  # get indexes
        return [item._label for item in self._items]


class Solution:
    def __init__(self, bins: list[Bin] = None, max_capacity: int = bin_capacity):
        if bins is None:
            bins = []
        self._bins = bins
        self._max_capacity = max_capacity
        self._fitness = self.calculate_fitness()

    def calculate_fitness(self):
        """somme(taille bin / max)² for each bin / nb bins"""
        '''if self._bins is []:
            return 1
        num_bins = len(self._bins)
        bins_capacity = np.asarray([_bin._capacity for _bin in self._bins], dtype=int)
        self._fitness = np.sum(np.power(bins_capacity / self._max_capacity, 2)) / num_bins
        return self._fitness'''
        self._fitness = len(self._bins) + sum((bin_capacity - bin._capacity / bin_capacity) for bin in self._bins) / (bin_capacity * len(self._bins))
        return self._fitness

    def count_items(self):
        return len([item for _bin in self._bins for item in _bin._items])

    def count_indexes(self):
        items = [item for _bin in self._bins for item in _bin._items]
        indexes = [item._label for item in items]
        return len(set(indexes))

    def affichage(self):
        return [bin.get_labels() for bin in self._bins]

    def __str__(self):
        return f"{self._fitness}: {affichage(self.affichage())}"



def next_fit(indexes: list[int]) -> Solution:
    random.shuffle(indexes)
    bins = []
    current_bin = Bin()
    for i in indexes:
        if current_bin._capacity + items[i] <= bin_capacity:
            current_bin.add_item(Item(label=i, size=items[i]))
        else:
            bins.append(current_bin)
            current_bin = Bin()
            current_bin.add_item(Item(label=i, size=items[i]))
    bins.append(current_bin)
    return Solution(bins)


def affichage(list_index: list[int]) -> list[int]:
    return [[items[i] for i in bin] for bin in list_index]


def generate_continent(n_teams: int) -> list[Solution]:
    indexes = list(items.keys())
    return np.array([next_fit(indexes) for _ in range(n_teams)], dtype=object)


def generate_preliminary(n_contients: int, n_teams: int) -> list[list[Solution]]:
    return np.array([generate_continent(n_teams) for _ in range(n_contients)], dtype=object)


"""def fitness(team: list[list[int]]):
    return len(team) + sum((bin_capacity - sum(bin) / bin_capacity) for bin in team) / (bin_capacity * len(team))"""




def crossover(father: Solution, mother: Solution):
    while True:
        two_points_father = np.random.randint(len(father._bins) + 1, size=2)
        two_points_mother = np.random.randint(len(mother._bins) + 1, size=2)
        if len(set(two_points_father)) == 2 and len(set(two_points_mother)) == 2:
            break
    two_points_father.sort()
    two_points_mother.sort()
    """choisir deux indexes for father and mother to do croisement a 2 points"""
    first_child = crossover_one_chromosome(chromosome=father, points_insert=two_points_father[1],
                                                genes=mother._bins[two_points_mother[0]:two_points_mother[1]])
    second_child = crossover_one_chromosome(chromosome=mother, points_insert=two_points_mother[0],
                                                genes=father._bins[two_points_father[0]:two_points_father[1]])
    return first_child, second_child


def crossover_one_chromosome(chromosome: Solution, points_insert: int, genes: list):
    """insert genes then completes with genes from chromosome"""
    child = Solution(genes, max_capacity=chromosome._max_capacity)
    take_labels = [_bin.get_labels() for _bin in genes]
    take_labels = [label for bin_labels in take_labels for label in bin_labels]
    left_bins = []
    right_bins = []
    free_items = []
    for i in range(len(chromosome._bins)):
        intersection = [label for label in chromosome._bins[i].get_labels() if label in take_labels]
        if len(intersection) == 0:
            if i < points_insert:
                left_bins.append(chromosome._bins[i])
            else:
                right_bins.append(chromosome._bins[i])
        else:
            for item in chromosome._bins[i]._items:
                if item._label not in intersection:
                    free_items.append(item)
    child._bins = left_bins + child._bins + right_bins
    return first_fit_descending(child, free_items)


def first_fit_descending(chromosome: Solution, items: list):
    """try to add leftover items in child's current bins else add new ones"""
    # chromosome = copy.deepcopy(chromosome)
    objects = copy.deepcopy(items)
    objects = sorted(objects, key=lambda item: item._size, reverse=True)
    max_capacity = chromosome._max_capacity
    add_bins = []
    current_bin = Bin()
    _flag = False
    _count = 0
    for item in objects:
        _count += 1
        for _bin in chromosome._bins:
            if _bin._capacity + item._size <= max_capacity:
                _bin.add_item(item)
                _flag = True
                break
        if not _flag:
            if current_bin._capacity + item._size <= max_capacity:
                current_bin.add_item(item)
            else:
                add_bins.append(current_bin)
                current_bin = Bin()
                current_bin.add_item(item)
        _flag = False
    if current_bin._capacity > 0:
        add_bins.append(current_bin)
    chromosome._bins += add_bins
    chromosome.calculate_fitness()
    return chromosome


def FIFA(n_iter: int, n_continents: int, n_teams: int):
    best_sol = None
    best_score = n
    start_time = time()

    preliminary = generate_preliminary(n_continents, n_teams)  # list of continents

    for p in range(n_iter):
        finals = []
        for continent in preliminary:
            team1, team2 = sorted(continent, reverse=False, key=lambda team: team._fitness)[:2]
            finals.extend([team1, team2])

        champion = sorted(finals, reverse=False, key=lambda team: team._fitness)[0]
        if champion._fitness < best_score:
            # print('found new best', champion._fitness, best_score, len(champion._bins))
            best_sol = champion
            best_score = champion._fitness

        next_preliminary = []
        print
        if p < n_iter - 1:
            for i in range(0, len(finals), 2):
                continent = finals[i: i + 2]
                for _ in range(0, n_teams - 2, 2):
                    continent.extend(crossover(continent[0], continent[1]))
                next_preliminary.append(continent)
            preliminary = next_preliminary
    # best_sol._bins.sort(reverse=True, key=lambda bin: bin._capacity if bin._capacity > bin_capacity // 2 else min(item._size for item in bin._items))

    fifi_time = time() - start_time
    return best_sol, fifi_time


# Define the simulated annealing function
def simulated_annealing(items, bin_capacity, initial_temperature, cooling_rate, nb_iter, n_iter: int, n_continents: int, n_teams: int):
    # Define the initial states
    # current_state = initial_state(items, bin_capacity)
    fifa_sol, duree_fifi = FIFA(n_iter, n_continents, n_teams)
    current_state = affichage(fifa_sol.affichage())
    current_energy = best_score = energy(current_state)
    best_state = current_state
    print("Start of RS (end of FIFA):")
    print("Start Score", best_score)
    print('Solution:', current_state)

    # Define the initial temperature
    current_temperature = initial_temperature
    start_time = time()

    # Loop until the temperature is too low
    while current_temperature > 1:
        # loop for nb_iter with same temperature
        for _ in range(nb_iter):
            # Pick a random neighbor state
            neighbor_state, diff = neighbor(current_state, bin_capacity)

            # Calculate the energy of the neighbor states
            neighbor_energy = energy(neighbor_state)

            # Calculate the energy difference
            energy_difference = (neighbor_energy - current_energy) * bin_capacity + diff

            # If the neighbor state has a lower energy, move to that state or with a certain probability
            if energy_difference < 0 or energy_difference > 0 and math.exp(-energy_difference / current_temperature) > random.random():
                current_state = neighbor_state
                current_energy = neighbor_energy

            # If current state better than the best one, update it
            if current_energy < best_score:
                best_state = current_state
                best_score = current_energy
        # Decrease the temperature
        current_temperature *= cooling_rate

    RS_time = time() - start_time
    # Return the best state found
    return best_state, RS_time + duree_fifi

# Define the initial state function
def initial_state(items, bin_capacity):
    # next fit wa9ila
    state = []
    bin = []
    for item in items:
        if sum(bin) + item <= bin_capacity:
            bin.append(item)
        else:
            state.append(bin)
            bin = [item]
    state.append(bin)
    return state

# Define the neighbor function
def neighbor(state, bin_capacity):
    i = random.randint(0, len(state)-1)  # random bin 1
    j = random.randint(0, len(state[i])-1)  # random object from bin 1
    k = random.randint(0, len(state)-1)  # random bin 2

    # if different bins and object from bin 1 can be added to bin 2, else do nothing
    if i != k and sum(state[k]) + state[i][j] <= bin_capacity:
        neighbor_state = copy.deepcopy(state)
        neighbor_state[k].append(neighbor_state[i].pop(j))
        if not neighbor_state[i]:  # remove bin 1 if it becomes empty
            neighbor_state.pop(i)
            return neighbor_state, 0
        return neighbor_state, sum(neighbor_state[i]) - sum(neighbor_state[k])
        # if nb of bins didn't change then we estimate the change by (i + object)² + k² - (i² + (k + object)²) = 2*object*(i - k) ~ i - k
        # the idea is to try to remplir un bin au maximum (min leftover space in a bin)
    return state, 0

# Define the energy function
def energy(state):
    return len(state)



#(n , bin_capacity , items ) = read_bpp_file('Hard28_BPP900.txt')  # optimal 75
#(n , bin_capacity , items ) = read_bpp_file('Hard28_BPP709.txt')  # optimal 67
#(n , bin_capacity , items ) = read_bpp_file('BPP_100_100_0.2_0.7_8.txt')  # optimal 46
(n , bin_capacity , items ) = read_bpp_file('BPP_50_200_0.1_0.8_4.txt')  # optimal 25
print(f"Nombre d'objets: {n}, capacité: {bin_capacity}\n")
items = {i:items[i] for i in range(n)}



# Test the algorithm
initial_temperature = n ** 2
cooling_rate = 0.999
nb_iter_RS = 5
nb_iter_FIFA = 50
nb_continents = 5
nb_teams = 4

best_state, duree_exe = simulated_annealing(list(items.values()), bin_capacity, initial_temperature, cooling_rate, nb_iter_RS, nb_iter_FIFA, nb_continents, nb_teams)
print("End of RS:")
print("End Score", len(best_state))
print('Solution:', best_state)
print("Execution time:", duree_exe)


Nombre d'objets: 50, capacité: 200

Start of RS (end of FIFA):
Start Score 26
Solution: [[93, 59, 48], [159, 21, 20], [115, 39, 21, 21], [123, 74], [159, 20, 20], [116, 50, 20], [56, 144], [120, 79], [55, 116, 20], [115, 82], [140, 55], [92, 108], [75, 114], [137, 62], [137, 61], [65, 114, 21], [26, 33, 33, 104], [134, 66], [148, 49], [153, 38], [147, 39], [158, 39], [128, 61], [94, 106], [141, 50], [139, 55]]
End of RS:
End Score 26
Solution: [[93, 59, 48], [159, 21, 20], [115, 39, 21, 21], [123, 74], [159, 20, 20], [116, 50, 20], [56, 144], [120, 79], [55, 116, 20], [115, 82], [140, 55], [92, 108], [75, 114], [137, 62], [137, 61], [65, 114, 21], [26, 33, 33, 104], [134, 66], [148, 49], [153, 38], [147, 39], [158, 39], [128, 61], [94, 106], [141, 50], [139, 55]]
Execution time: 0.1774885654449463




```
# This is formatted as code
```

# FIFA ALONE

In [62]:
import random
import math
import copy
import numpy as np
from time import time


def read_bpp_file(file_path):
    with open(file_path, 'r') as f:
        n = int(f.readline().strip())
        capacity = int(f.readline().strip())
        objects = [int(line.strip()) for line in f]
    return (n, capacity, objects)



#(n , bin_capacity , items ) = read_bpp_file('Hard28_BPP900.txt')  # optimal 75
#(n , bin_capacity , items ) = read_bpp_file('benchmark180.txt')  # optimal 67
(n , bin_capacity , items ) = read_bpp_file('benchmark100.txt')  # optimal 46
#(n , bin_capacity , items ) = read_bpp_file('benchmark50.txt')  # optimal 25


class Item:
    def __init__(self, label: int = None, size: int = None):
        self._label = label  # index
        self._size = size  # weight


class Bin:
    def __init__(self, items: list = None, capacity: int = 0):
        if items is None:
            items = []
        self._items = items
        self._capacity = capacity

    def add_item(self, item: Item):
        self._items.append(item)
        self._capacity += item._size

    def get_labels(self) -> list[int]:  # get indexes
        return [item._label for item in self._items]


class Solution:
    def __init__(self, bins: list[Bin] = None, max_capacity: int = bin_capacity):
        if bins is None:
            bins = []
        self._bins = bins
        self._max_capacity = max_capacity
        self._fitness = self.calculate_fitness()

    def calculate_fitness(self):
        """somme(taille bin / max)² for each bin / nb bins"""
        '''if self._bins is []:
            return 1
        num_bins = len(self._bins)
        bins_capacity = np.asarray([_bin._capacity for _bin in self._bins], dtype=int)
        self._fitness = np.sum(np.power(bins_capacity / self._max_capacity, 2)) / num_bins
        return self._fitness'''
        self._fitness = len(self._bins) + 1 - sum((bin._capacity / bin_capacity) ** 2 for bin in self._bins) / (bin_capacity * len(self._bins)) ** 2
        return self._fitness

    def count_items(self):
        return len([item for _bin in self._bins for item in _bin._items])

    def count_indexes(self):
        items = [item for _bin in self._bins for item in _bin._items]
        indexes = [item._label for item in items]
        return len(set(indexes))

    def affichage(self):
        return [bin.get_labels() for bin in self._bins]

    def __str__(self):
        return f"{self._fitness}: {affichage(self.affichage())}"



def next_fit(indexes: list[int]) -> Solution:
    random.shuffle(indexes)
    bins = []
    current_bin = Bin()
    for i in indexes:
        if current_bin._capacity + items[i] <= bin_capacity:
            current_bin.add_item(Item(label=i, size=items[i]))
        else:
            bins.append(current_bin)
            current_bin = Bin()
            current_bin.add_item(Item(label=i, size=items[i]))
    bins.append(current_bin)
    return Solution(bins)


def affichage(list_index: list[int]) -> list[int]:
    return [[items[i] for i in bin] for bin in list_index]


def generate_continent(n_teams: int) -> list[Solution]:
    indexes = list(items.keys())
    return np.array([next_fit(indexes) for _ in range(n_teams)], dtype=object)


def generate_preliminary(n_contients: int, n_teams: int) -> list[list[Solution]]:
    return np.array([generate_continent(n_teams) for _ in range(n_contients)], dtype=object)


def crossover(father: Solution, mother: Solution):
    while True:
        two_points_father = np.random.randint(len(father._bins) + 1, size=2)
        two_points_mother = np.random.randint(len(mother._bins) + 1, size=2)
        if len(set(two_points_father)) == 2 and len(set(two_points_mother)) == 2:
            break
    two_points_father.sort()
    two_points_mother.sort()
    """choisir deux indexes for father and mother to do croisement a 2 points"""
    first_child = crossover_one_chromosome(chromosome=father, points_insert=two_points_father[1],
                                                genes=mother._bins[two_points_mother[0]:two_points_mother[1]])
    second_child = crossover_one_chromosome(chromosome=mother, points_insert=two_points_mother[0],
                                                genes=father._bins[two_points_father[0]:two_points_father[1]])
    return first_child, second_child


def crossover_one_chromosome(chromosome: Solution, points_insert: int, genes: list):
    """insert genes then completes with genes from chromosome"""
    child = Solution(genes, max_capacity=chromosome._max_capacity)
    take_labels = [_bin.get_labels() for _bin in genes]
    take_labels = [label for bin_labels in take_labels for label in bin_labels]
    left_bins = []
    right_bins = []
    free_items = []
    for i in range(len(chromosome._bins)):
        intersection = [label for label in chromosome._bins[i].get_labels() if label in take_labels]
        if len(intersection) == 0:
            if i < points_insert:
                left_bins.append(chromosome._bins[i])
            else:
                right_bins.append(chromosome._bins[i])
        else:
            for item in chromosome._bins[i]._items:
                if item._label not in intersection:
                    free_items.append(item)
    child._bins = left_bins + child._bins + right_bins
    return first_fit_descending(child, free_items)


def first_fit_descending(chromosome: Solution, items: list):
    """try to add leftover items in child's current bins else add new ones"""
    # chromosome = copy.deepcopy(chromosome)
    objects = copy.deepcopy(items)
    objects = sorted(objects, key=lambda item: item._size, reverse=True)
    max_capacity = chromosome._max_capacity
    add_bins = []
    current_bin = Bin()
    _flag = False
    _count = 0
    for item in objects:
        _count += 1
        for _bin in chromosome._bins:
            if _bin._capacity + item._size <= max_capacity:
                _bin.add_item(item)
                _flag = True
                break
        if not _flag:
            if current_bin._capacity + item._size <= max_capacity:
                current_bin.add_item(item)
            else:
                add_bins.append(current_bin)
                current_bin = Bin()
                current_bin.add_item(item)
        _flag = False
    if current_bin._capacity > 0:
        add_bins.append(current_bin)
    chromosome._bins += add_bins
    chromosome.calculate_fitness()
    return chromosome


def FIFA(n_iter: int, n_continents: int, n_teams: int):
    best_sol = None
    best_score = n
    start_time = time()

    preliminary = generate_preliminary(n_continents, n_teams)  # list of continents

    for p in range(n_iter):
        finals = []
        for continent in preliminary:
            team1, team2 = sorted(continent, reverse=False, key=lambda team: team._fitness)[:2]
            finals.extend([team1, team2])

        champion = sorted(finals, reverse=False, key=lambda team: team._fitness)[0]
        if champion._fitness < best_score:
            print('found new best', champion._fitness, best_score)
            best_sol = champion
            best_score = champion._fitness

        next_preliminary = []
        print
        if p < n_iter - 1:
            for i in range(0, len(finals), 2):
                continent = finals[i: i + 2]
                for _ in range(0, n_teams - 2, 2):
                    continent.extend(crossover(continent[0], continent[1]))
                next_preliminary.append(continent)
            preliminary = next_preliminary
    # best_sol._bins.sort(reverse=True, key=lambda bin: bin._capacity if bin._capacity > bin_capacity // 2 else min(item._size for item in bin._items))

    fifi_time = time() - start_time
    return best_sol, fifi_time


print(f"Nombre d'objets: {n}, capacité: {bin_capacity}\n")
items = {i:items[i] for i in range(n)}


# Test the algorithm
nb_iter_FIFA = 50
nb_continents = 5
nb_teams = 4

best_sol, duree_exe = FIFA(nb_iter_FIFA, nb_continents, nb_teams)
best_sol = affichage(best_sol.affichage())
print("End Score", len(best_sol))
print('Solution:', best_sol)
print("Execution time:", duree_exe)


Nombre d'objets: 100, capacité: 100

found new best 60.999999014661114 100
found new best 53.99999859204699 60.999999014661114
found new best 51.99999844590542 53.99999859204699
found new best 51.99999841014994 51.99999844590542
found new best 51.999998282456744 51.99999841014994
found new best 51.9999982767128 51.999998282456744
found new best 50.999998226712 51.9999982767128
found new best 49.99999818646814 50.999998226712
End Score 49
Solution: [[47, 53], [51, 36], [56, 35], [46, 26, 26], [43, 23, 23], [64, 25], [44, 54], [59, 34], [51, 43], [63, 36], [49, 38], [65, 28], [67, 25], [65, 33], [53, 47], [54, 38], [57, 35], [65, 20], [41, 59], [37, 58], [64, 27], [20, 58, 22], [47, 46], [36, 28, 26], [59, 40], [59, 37], [68, 24], [63, 37], [52, 41], [65, 29], [58, 41], [51, 48], [51, 49], [62, 29], [60, 36], [52, 46], [65, 26], [57, 43], [53, 41], [47, 52], [58, 30], [60, 31], [70, 30], [24, 43, 23], [63, 20], [68, 30], [67, 29], [62, 20], [54, 44]]
Execution time: 0.10693812370300293


# FIFA + WHALES

In [ ]:
import random
import math
import copy
import numpy as np
from time import time



def read_bpp_file(file_path):
    with open(file_path, 'r') as f:
        n = int(f.readline().strip())
        capacity = int(f.readline().strip())
        objects = [int(line.strip()) for line in f]
    return (n, capacity, objects)


#(n , bin_capacity , items ) = read_bpp_file('Hard28_BPP900.txt')  # optimal 75
(n , bin_capacity , items ) = read_bpp_file('Hard28_BPP709.txt')  # optimal 67
#(n , bin_capacity , items ) = read_bpp_file('BPP_100_100_0.2_0.7_8.txt')  # optimal 46
#(n , bin_capacity , items ) = read_bpp_file('BPP_50_200_0.1_0.8_4.txt')  # optimal 25
print(f"Nombre d'objets: {n}, capacité: {bin_capacity}\n")



# Test the algorithm
nb_iter_FIFA = 50
nb_continents = 5
nb_teams = 4
num_whales = nb_continents * nb_teams
num_items = n



class Item:
    def __init__(self, label: int = None, size: int = None):
        self._label = label  # index
        self._size = size  # weight


class Bin:
    def __init__(self, items: list = None, capacity: int = 0):
        if items is None:
            items = []
        self._items = items
        self._capacity = capacity

    def add_item(self, item: Item):
        self._items.append(item)
        self._capacity += item._size

    def get_labels(self) -> list[int]:  # get indexes
        return [item._label for item in self._items]


class Solution:
    def __init__(self, bins: list[Bin] = None, max_capacity: int = bin_capacity):
        if bins is None:
            bins = []
        self._bins = bins
        self._max_capacity = max_capacity
        self._fitness = self.calculate_fitness()

    def calculate_fitness(self):
        """somme(taille bin / max)² for each bin / nb bins"""
        '''if self._bins is []:
            return 1
        num_bins = len(self._bins)
        bins_capacity = np.asarray([_bin._capacity for _bin in self._bins], dtype=int)
        self._fitness = np.sum(np.power(bins_capacity / self._max_capacity, 2)) / num_bins
        return self._fitness'''
        self._fitness = len(self._bins) + sum((bin_capacity - bin._capacity / bin_capacity) for bin in self._bins) / (bin_capacity * len(self._bins))
        return self._fitness

    def count_items(self):
        return len([item for _bin in self._bins for item in _bin._items])

    def count_indexes(self):
        items = [item for _bin in self._bins for item in _bin._items]
        indexes = [item._label for item in items]
        return len(set(indexes))

    def affichage(self):
        return [bin.get_labels() for bin in self._bins]

    def __str__(self):
        return f"{self._fitness}: {affichage(self.affichage())}"



"""def next_fit(indexes: list[int]) -> Solution:
    random.shuffle(indexes)
    bins = []
    current_bin = Bin()
    for i in indexes:
        if current_bin._capacity + items[i] <= bin_capacity:
            current_bin.add_item(Item(label=i, size=items[i]))
        else:
            bins.append(current_bin)
            current_bin = Bin()
            current_bin.add_item(Item(label=i, size=items[i]))
    bins.append(current_bin)
    return Solution(bins)"""


def affichage(list_index: list[int]) -> list[int]:
    return [[items[i] for i in bin] for bin in list_index]


"""def generate_continent(n_teams: int) -> list[Solution]:
    indexes = list(items.keys())
    return np.array([next_fit(indexes) for _ in range(n_teams)], dtype=object)"""


def generate_preliminary(n_contients: int, n_teams: int) -> list[list[Solution]]:
    global items

    population = initialize_population(num_whales, items, bin_capacity)
    solb, pop = opt(population)

    items = {i:items[i] for i in range(n)}
    continents = []
    continent = []
    for i, sol in enumerate(pop):
        team = []
        for bin in sol:
            current_bin = Bin()
            for item in bin:
                current_bin.add_item(Item(label=item, size=items[item]))
            assert current_bin._capacity <= bin_capacity
            team.append(current_bin)
        if i != 0 and i % n_teams == 0:
            continents.append(continent)
        else:
            continent.append(Solution(team))
    return np.array(continents, dtype=object)


def crossover(father: Solution, mother: Solution):
    while True:
        two_points_father = np.random.randint(len(father._bins) + 1, size=2)
        two_points_mother = np.random.randint(len(mother._bins) + 1, size=2)
        if len(set(two_points_father)) == 2 and len(set(two_points_mother)) == 2:
            break
    two_points_father.sort()
    two_points_mother.sort()
    """choisir deux indexes for father and mother to do croisement a 2 points"""
    first_child = crossover_one_chromosome(chromosome=father, points_insert=two_points_father[1],
                                                genes=mother._bins[two_points_mother[0]:two_points_mother[1]])
    second_child = crossover_one_chromosome(chromosome=mother, points_insert=two_points_mother[0],
                                                genes=father._bins[two_points_father[0]:two_points_father[1]])
    return first_child, second_child


def crossover_one_chromosome(chromosome: Solution, points_insert: int, genes: list):
    """insert genes then completes with genes from chromosome"""
    child = Solution(genes, max_capacity=chromosome._max_capacity)
    take_labels = [_bin.get_labels() for _bin in genes]
    take_labels = [label for bin_labels in take_labels for label in bin_labels]
    left_bins = []
    right_bins = []
    free_items = []
    for i in range(len(chromosome._bins)):
        intersection = [label for label in chromosome._bins[i].get_labels() if label in take_labels]
        if len(intersection) == 0:
            if i < points_insert:
                left_bins.append(chromosome._bins[i])
            else:
                right_bins.append(chromosome._bins[i])
        else:
            for item in chromosome._bins[i]._items:
                if item._label not in intersection:
                    free_items.append(item)
    child._bins = left_bins + child._bins + right_bins
    return first_fit_descending(child, free_items)


def first_fit_descending(chromosome: Solution, items: list):
    """try to add leftover items in child's current bins else add new ones"""
    # chromosome = copy.deepcopy(chromosome)
    objects = copy.deepcopy(items)
    objects = sorted(objects, key=lambda item: item._size, reverse=True)
    max_capacity = chromosome._max_capacity
    add_bins = []
    current_bin = Bin()
    _flag = False
    _count = 0
    for item in objects:
        _count += 1
        for _bin in chromosome._bins:
            if _bin._capacity + item._size <= max_capacity:
                _bin.add_item(item)
                _flag = True
                break
        if not _flag:
            if current_bin._capacity + item._size <= max_capacity:
                current_bin.add_item(item)
            else:
                add_bins.append(current_bin)
                current_bin = Bin()
                current_bin.add_item(item)
        _flag = False
    if current_bin._capacity > 0:
        add_bins.append(current_bin)
    chromosome._bins += add_bins
    chromosome.calculate_fitness()
    return chromosome


def FIFA(n_iter: int, n_continents: int, n_teams: int):
    best_sol = None
    best_score = n
    start_time = time()

    preliminary = generate_preliminary(n_continents, n_teams)  # list of continents

    for p in range(n_iter):
        finals = []
        for continent in preliminary:
            team1, team2 = sorted(continent, reverse=False, key=lambda team: team._fitness)[:2]
            finals.extend([team1, team2])

        champion = sorted(finals, reverse=False, key=lambda team: team._fitness)[0]
        if champion._fitness < best_score:
            # print('found new best', champion._fitness, best_score, len(champion._bins))
            best_sol = champion
            best_score = champion._fitness

        next_preliminary = []
        print
        if p < n_iter - 1:
            for i in range(0, len(finals), 2):
                continent = finals[i: i + 2]
                for _ in range(0, n_teams - 2, 2):
                    continent.extend(crossover(continent[0], continent[1]))
                next_preliminary.append(continent)
            preliminary = next_preliminary
    # best_sol._bins.sort(reverse=True, key=lambda bin: bin._capacity if bin._capacity > bin_capacity // 2 else min(item._size for item in bin._items))
    fifi_time = time() - start_time
    return best_sol, fifi_time




def Levyflight():
    beta = 1.5
    f1 = math.gamma(1+beta)
    f2 = beta * math.gamma(1+beta) / 2
    f3 = np.sin(np.pi*beta/2)
    f4 = 2**( (beta-1)/2 )
    sigma_u = ( f1/f2 * f3/f4 ) ** (2/beta)
    sigma_v = 1.0 # (12)

    u = np.random.normal(0, sigma_u)
    v = np.random.normal(0, sigma_v)
    s = u / ( np.abs(v)**(1/beta) )

    return s

def initialize_population(num_whales, items, bin_capacity):
    population = []
    for _ in range(num_whales):
        whale = []
        bin = []
        bin_pos = []
        bin_capacity_remaining = bin_capacity
        items_pos = [i for i in range(1, num_items+1)]
        random.shuffle(items_pos)  # Shuffle the items randomly
        whale , fitness = decode_FF(items,items_pos,bin_capacity)
        whale = Whale(whale, fitness , items_pos)  # Create an instance of whale
        population.append(whale)
    return population

def best_agent(population):
    best = min(population, key=lambda obj: obj.fitness)
    return best

def fix_pos(X):
    list = []
    list_t = [i for i in range(1,num_items+1)]
    for xd in X :
        if xd not in list :
            list.append(xd)
            list_t.remove(xd)
        else :
            list.append(list_t[0])
            list_t.remove(list_t[0])
    return list


def levy_order_value(solution):
    sorted_indices = np.argsort(solution)[::-1]  # Sort the solution indices in descending order
    discrete_solution = np.zeros_like(solution)
    for i, index in enumerate(sorted_indices):
        discrete_solution[index] = i + 1  # Assign a discrete value based on the index position
    return discrete_solution


def apply_mutation(array):
    if len(array) == 0:
        return array  # Return the array as is if it's empty

    # Step 1: Select two random indices within the range of valid indices
    index1 = random.randint(0, len(array) - 1)
    index2 = random.randint(0, len(array) - 1)

    # Step 2: Swap the values at the two random indices
    array[index1], array[index2] = array[index2], array[index1]

    # Step 3: Shift the items by a random number of steps
    num_steps = random.randint(2, len(array) - 2)
    shifted_array = shift_array(array, num_steps)

    return shifted_array

def shift_array(array, num_steps):
    num_steps = num_steps % len(array)  # Ensure steps are within valid range

    shifted_array = array[num_steps:] + array[:num_steps]

    return shifted_array

def fitnessC(pos , items=items,capacity = bin_capacity):
    whale , fitness = decode_FF(items,pos,bin_capacity)
    return whale,fitness

def decode_FFC(items,position,bin_capacity):
    bin =[]
    bins = []
    bin_capacity_remaining = bin_capacity
    for i in position :
        if(items[i-1] <= bin_capacity_remaining):
            bin.append(i-1)
            bin_capacity_remaining -= items[i-1]
        else:
            bins.append(bin)
            bin_capacity_remaining = bin_capacity
            bin = [i-1]
            bin_capacity_remaining -= items[i-1]
    if bin :
        bins.append(bin)
    return bins , len(bins)


def decode_FF(items,position,bin_capacity):
    bin =[]
    bins = []
    bin_capacity_remaining = bin_capacity
    for i in position :
        if(items[i-1] <= bin_capacity_remaining):
            bin.append(items[i-1])
            bin_capacity_remaining -= items[i-1]
        else:
            bins.append(bin)
            bin_capacity_remaining = bin_capacity
            bin = [items[i-1]]
            bin_capacity_remaining -= items[i-1]
    if bin :
        bins.append(bin)
    return bins , len(bins)

class Whale:
    def __init__(self, objects, fitness , pos):
        self.objects = objects
        self.fitness = fitness
        self.pos = pos

def opt(population):
    ub=num_items
    lb=1,
    b=1
    a_max=2
    a_min=0
    a2_max=-1
    a2_min=-2
    l_max=1
    l_min=-1
    runtimes  = 5
    best = best_agent(population)
    best_X = best.pos
    best_F = best.fitness
    pop = []
    for whale in population :
        X = whale.pos
        F = whale.fitness
        if F < best_F:
            best_X = best.pos.copy()
            best_F = F
        a = a_max - (a_max-a_min)*(0.01)
        a2 = a2_max - (a2_max-a2_min)*(0.01)
        p = np.random.uniform()
        r1 = np.random.uniform()
        r2 = np.random.uniform()
        r3 = np.random.uniform()
        A = 2*a*r1 - a #(2.3)
        C = 2*r2 # (2.4)
        l = (a2-1)*r3 + 1 # (???)

        if p>0.5:
            D = np.abs(np.array(best_X) - np.array(X))
            X = D*np.exp(b*l)*np.cos(2*np.pi*l)+best_X# (6)
        else:
            if np.abs(A)<1:
                C_arr = np.full_like(X, C)
                D = np.abs(C_arr*best_X - X)
                X = best_X - A*D # (2)
            else:
                random_index = random.randint(0,num_whales-1)
                X_rand = population[random_index].pos
                C_arr = np.full_like(X_rand, C)
                D = np.abs(C_arr*X_rand - X)
                X = X_rand - A*D # (9)


        u = np.random.uniform()
        r4 = np.random.uniform()
        X = X +u*np.sign(r4-0.5)*Levyflight()

        X = np.clip(X, lb, ub)
        discrete = levy_order_value(X).tolist()
        X = [int(xd) for xd in X]
        objs ,fitness = decode_FF(items,X,bin_capacity)
        whale.pos = discrete
        whale.objects = objs
        whale.fitness = fitness
        if fitness < best_F :
            best_F = fitness
            best_X = X
        else :
            best_X = apply_mutation(best_X)
            best_X = [int(xd) for xd in best_X]
            # print(best_X)
            _,F = fitnessC(best_X)
        pop.append(decode_FFC(items,fix_pos(X),bin_capacity)[0])
    return fix_pos(best_X) , pop




best_sol, duree_exe = FIFA(nb_iter_FIFA, nb_continents, nb_teams)
best_sol = affichage(best_sol.affichage())
print("End Score", len(best_sol))
print('Solution:', best_sol)
print("Execution time:", duree_exe)


Nombre d'objets: 180, capacité: 1000

End Score 69
Solution: [[661, 339], [616, 361], [516, 454, 27], [555, 444], [552, 288, 153], [543, 364, 88], [357, 274, 301, 63], [543, 211, 222], [535, 281, 178], [532, 272, 113, 76], [202, 189, 189, 189, 183, 44], [606, 334, 45], [601, 326, 67], [599, 323, 76], [573, 270, 145], [565, 116, 105, 180, 27], [584, 145, 267], [530, 253, 170, 38], [528, 465], [509, 491], [506, 475], [425, 423, 152], [636, 310, 46], [630, 270, 77], [517, 481], [514, 443, 38], [508, 489], [432, 428, 137], [499, 498], [474, 470, 50], [677, 321], [646, 346], [680, 264, 46, 9], [673, 228, 98], [671, 198, 105], [663, 198, 139], [650, 343], [644, 341], [626, 372], [624, 368], [420, 400, 172], [697, 292], [593, 407], [682, 310], [664, 325], [421, 420, 155], [688, 308], [687, 306], [676, 288, 27], [608, 390], [512, 453], [511, 455, 16, 16], [579, 410], [525, 464], [299, 578, 106, 16], [578, 409], [570, 202, 222], [405, 379, 202], [532, 275, 177, 14], [608, 356], [566, 421], [439